In [ ]:
# This will import the autoreload extension
%load_ext autoreload

# This will set autoreload to reload all modules (excluding those that can't be reloaded)
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from SARLens.processor.focus import coarseRDA
from SARLens.utils.io import load, plot_with_cdf
import torch
import torch.nn.functional as F  

# setting device:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Focusing:

#### Decoding

In [ ]:
input_file = '/Data_large/marine/PythonProjects/AutoFocusNet/Data/SANPAOLO/S1A_S3_RAW__0SDH_20240524T213606_20240524T213631_054018_069139_241A.SAFE/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139.dat'
output_folder = '/Data_large/marine/PythonProjects/AutoFocusNet/Data/Decoded/SANPAOLO'
!python -m SARLens.processor.decode -i {input_file} -o {output_folder}

### RDA:

In [ ]:
data_path = '/Data_large/marine/PythonProjects/AutoFocusNet/Data/Decoded/SANPAOLO'

radar_data = load(f'{data_path}/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_pkt_0.pkl')
metadata = load(f'{data_path}/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_pkt_0_metadata.pkl')
ephemeris = load(f'{data_path}/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_ephemeris.pkl')

# init
radar_data = torch.from_numpy(radar_data).to("cpu")

RadarProcessor = coarseRDA(
    raw_data={'echo':radar_data[:,:], 'ephemeris':ephemeris, 'metadata':metadata}, 
    verbose=False, 
    backend='torch'
)

RadarProcessor.data_focus()

In [ ]:
plot_with_cdf(RadarProcessor.radar_data, (24,24))